# Mini project I

Content:
- I/O
- Featurisation
- Dimension reduction
- MSM estimation
- MSM validation

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mdshare
import pyemma
import deeptime as dt

pdb = mdshare.fetch('pentapeptide-impl-solv.pdb', working_directory='data')
files = mdshare.fetch('pentapeptide-*-500ns-impl-solv.xtc', working_directory='data')

Load two different molecular features:
- backbone torsions (with `cossin=True` and `periodic=False`)
- backbone atom positions (with `periodic=False`)

In [ ]:
torsions_feat = pyemma.coordinates.featurizer(pdb)
torsions_feat. #FIXME
torsions_data = #FIXME

positions_feat = pyemma.coordinates.featurizer(pdb)
positions_feat. #FIXME
positions_data = #FIXME

Compute VAMP scores for each molecular feature at a lag time of five steps and a dimensionality of four:

In [ ]:
torsions_scores = #FIXME
print(f'Torsions:  {np.mean(torsions_scores):.2f}±{np.std(torsions_scores):.2f}')

positions_scores = #FIXME
print(f'Positions: {np.mean(positions_scores):.2f}±{np.std(positions_scores):.2f}')

Perform a dimension reduction of the torsion data with TICA at a lag time of five steps and four independent components:

In [ ]:
tica = #FIXME
tica_output = #FIXME

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(
    np.concatenate(tica_output),
    ax=axes[0],
    feature_labels=['IC1', 'IC2', 'IC3', 'IC4'],
    ylog=True)
pyemma.plots.plot_density(*np.concatenate(tica_output)[:, :2].T, ax=axes[1], logscale=True)
axes[1].set_xlabel('IC 1')
axes[1].set_ylabel('IC 2')
fig.tight_layout()

Discretise with 75 $k$-means centers and a stride of 10 and show the free energy surface for the first two independent components:

In [ ]:
cluster = #FIXME
dtrajs = #FIXME

fig, ax = plt.subplots(figsize=(4, 4))
pyemma.plots. #FIXME
ax.scatter(*cluster.cluster_centers[:, :2].T, s=15, c='k')
ax.set_xlabel('IC 1')
ax.set_ylabel('IC 2')
fig.tight_layout()

Plot the first 10 implied timescales with errorbars up to a lag time of 50 steps (trajectory spacing is $0.1\frac{\mathrm{ns}}{\mathrm{step}}$):

In [ ]:
its = #FIXME
pyemma.plots. #FIXME

Estimate a Bayesian MSM at a lag time of five steps and show a CK test for five metastable states:

In [ ]:
msm = #FIXME

pyemma.plots. #FIXME